## Web Scraping Using Selenium for "Flight Price Prediction Project"

In [1]:
                                           ''' Importing required libraries'''
    
import selenium
from bs4 import BeautifulSoup
import time
import pandas as pd


# Importing selenium webdriver
from selenium import webdriver

# Importing Exceptions
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException, ElementNotVisibleException

# importing requests
import requests

#importing regex
import re

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
# Activating the chrome browser
driver = webdriver.Chrome(r"C:/Users/Rohit/Documents/New/chromedriver.exe") 
time.sleep(2)

# opening yatra.com
driver.get('https://www.yatra.com/')
time.sleep(2)

# searching flights
search_btn = driver.find_element_by_xpath('//input[@value="Search Flights"]')
search_btn.click()
time.sleep(1)


# creating a list of cities to see flight tickets
cities=['Mumbai', 'Bangalore', 'Pune', 'Chennai', 'Hyderabad', 'Goa', 'Kolkata', 'Patna', 'Jaipur']


# Creating empty lists to store the values
source_city = []
destination_city = []
airline_name=[]
departure_time = []
arrival_time = []
duration=[]
stops = []
source = []
destination = []
meal=[]
loc=[]
price=[]


for a in cities:
    for b in cities:
        if a!=b:            
            source_city = driver.find_element_by_xpath('//input[@id="origin_0"]')
            time.sleep(2)
            source_city.clear() 
            source_city.send_keys(a)
            time.sleep(2)
            
            destination_city = driver.find_element_by_xpath('//input[@id="destination_0"]')
            time.sleep(2)
            destination_city.click()
            destination_city.clear()
            time.sleep(2)
            destination_city.send_keys(b)
            time.sleep(2)
            
            try:
                search_button = driver.find_element_by_xpath("//button[@class='fs-14 btn-submit cursor-pointer bold']")
                wait = WebDriverWait(driver, 10)
                wait.until(EC.visibility_of(search_button))
                search_button.click()
            except NoSuchElementException:
                    pass
            time.sleep(1)
            
            #scrolling the page
            for _ in range(15):
                time.sleep(1)
                driver.execute_script("window.scrollBy(0,5000)")
                time.sleep(5)
            #clicking on the button to go on top of the page
            try:
                driver.find_element_by_xpath("//div[@title='scroll to top']").click()
            except NoSuchElementException:
                    pass
            
           #fetching name of airline
            try:
                for i in driver.find_elements_by_xpath('//span[@class="i-b text ellipsis"]'):
                    airline_name.append(i.text)
            except NoSuchElementException:
                    pass
            
            #time of depart
            try:
                for i in driver.find_elements_by_xpath('//div[@class="i-b pr"]'):
                    departure_time.append(i.text)
            except NoSuchElementException:
                    pass
            
            #time of arrival
            try:
                for i in driver.find_elements_by_xpath('//p[@class="bold fs-15 mb-2 pr time"]'):
                    arrival_time.append(i.text)
                    #arrival_time.append(i.get_attribute("innerHTML").splitlines()[0][0:5])
            except NoSuchElementException:
                    pass
            
            #duration
            try:
                for i in driver.find_elements_by_xpath('//p[@class="fs-12 bold du mb-2"]'):
                    duration.append(i.text)
            except NoSuchElementException:
                    pass
            
            #number of stops
            try:
                for i in driver.find_elements_by_xpath('//div[@class=" font-lightgrey fs-10 tipsy i-b fs-10"]'):
                    stops.append(i.text)
            except NoSuchElementException:
                    pass
            
            #locations
            try:
                for i in driver.find_elements_by_xpath("//p[@class='fs-10 font-lightgrey no-wrap city ellipsis']"):
                    loc.append(i.text)
            except NoSuchElementException:
                    pass
                
            #fetching meal
            try:
                for i in driver.find_elements_by_xpath("//div[@class='features pull-right fs-12 flex']/div[1]"):
                    if (i.text) == 'Free Meal':
                        meal.append('Free Meal')
                    else:
                        meal.append('Paid Meal')
            except NoSuchElementException:
                    pass
            
            #fetching prices
            try:
                for i in driver.find_elements_by_xpath('//div[@class="i-b tipsy fare-summary-tooltip fs-18"]'):
                    price.append(i.text)
            except NoSuchElementException:
                    pass

In [3]:
print(len(airline_name),len(departure_time),len(arrival_time),len(duration),len(stops),len(meal),len(price))

4547 4547 4547 4547 4547 4532 4547


In [138]:
#removing unwanted data from "arrival_time"
arrival_time = [i[:5] for i in arrival_time]

#removing "(s)" from every stops
stops = [i.replace('(s)','s') for i in stops]

In [139]:
source = loc[0:len(loc):2]
destination = loc[1:len(loc):2]

In [140]:
len(source)

4547

In [141]:
len(destination)

4547

In [145]:
                                      ''' Creating a Dataframe for 4500 flight data '''

df = pd.DataFrame({"Airline Name":airline_name[:4500], "Source":source[:4500], "Destination":destination[:4500], "Departure Time":departure_time[:4500], "Arrival Time":arrival_time[:4500], "Duration":duration[:4500], "Total Stops":stops[:4500], "Price":price[:4500]})
df

,Airline Name,Source,Destination,Departure Time,Arrival Time,Duration,Total Stops,Price
0,Air India,New Delhi,Mumbai,07:00,09:05,2h 05m,Non Stop,"6,690"
1,IndiGo,New Delhi,Mumbai,07:15,09:25,2h 10m,Non Stop,"6,690"
2,Air India,New Delhi,Mumbai,08:00,10:10,2h 10m,Non Stop,"6,690"
3,IndiGo,New Delhi,Mumbai,08:10,10:20,2h 10m,Non Stop,"6,690"
4,Go First,New Delhi,Mumbai,11:15,13:25,2h 10m,Non Stop,"6,690"
...,...,...,...,...,...,...,...,...
4495,Air India,Jaipur,Kolkata,20:35,22:20,25h 45m,1 Stop,"13,616"
4496,IndiGo,Jaipur,Kolkata,09:55,20:15,10h 20m,1 Stop,"13,721"
4497,IndiGo,Jaipur,Kolkata,09:55,22:00,12h 05m,1 Stop,"13,721"
4498,IndiGo,Jaipur,Kolkata,14:20,21:55,7h 35m,1 Stop,"13,826"


In [146]:
df.to_csv('Flight_price_data', index=False)

In [147]:
driver.close()

                                             ----- --:-- ---- --:-- ---- --:-- -----